In [ ]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)
import os
import model
import detect_utils
import cv2
import json
import matplotlib.pyplot as plt
from coco_names import COCO_INSTANCE_CATEGORY_NAMES



In [ ]:

print("Torch version:", torch.__version__)

# create own Dataset
training_dataset = myOwnDataset(
    root=config.data_dir, annotation=config.train_coco, transforms=get_transform()
)

# training DataLoader
training_loader = torch.utils.data.DataLoader(
    training_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in training_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]


model = model.get_model(device = device, model_name='v2')


# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(training_loader)


In [ ]:
loss_per_epoch = []

# Training
for epoch in range(config.num_epochs):
#for epoch in range(1):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    
    sum_loss = 0
    loss_per_epoch = []

# Training
for epoch in range(config.num_epochs):
#for epoch in range(1):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    
    sum_loss = 0
    for imgs, annotations in training_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        # Check if there are any boxes in the annotations
        has_boxes = any(anno['boxes'].size(0) > 0 for anno in annotations)

        # If there are no boxes in any of the images in the batch, skip the iteration
        if not has_boxes:
            continue

        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")
        sum_loss += losses
    
    #print average loss for epoch
    print(f"Average loss for epoch: {sum_loss/len_dataloader}")

    loss_per_epoch.append(sum_loss/len_dataloader)

In [ ]:
#load json file

with open(config.test_coco) as f:
    json_data = json.load(f)



num_correct = 0
total = 0
# create test Dataset
testing_dataset = myOwnDataset(
    root=config.data_dir, annotation=config.test_coco, transforms=get_transform()
)
#testing DataLoader
testing_loader = torch.utils.data.DataLoader(
    testing_dataset,
    batch_size=config.test_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


model.eval()

# Evaluate the model
with torch.no_grad():
    num_correct_boxes = 0
    num_correct_labels = 0
    total_boxes = 0

    for imgs, annotations in testing_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        for idx, image in enumerate(imgs):
            
            image_np = image.cpu().numpy().transpose(1, 2, 0)  # Convert image to NumPy array with the correct shape
            #print(idx)

            # plot the image
            #plt.imshow(image_np)

            # show the plot
            #plt.show()
            boxes, pred_classes, labels = detect_utils.predict(image_np, model, device, 0.4)
            print("labels: ")
            print(pred_classes)
            #print("Unique label values:", torch.unique(labels).cpu().numpy())

            gt_boxes = annotations[idx]['boxes'].cpu().numpy()
            gt_labels = annotations[idx]['labels'].cpu().numpy()

            correct_labels = []
            
            for label_index in gt_labels:
                    if (label_index == 32):
                        label_index = 0
                    correct_labels.append(COCO_INSTANCE_CATEGORY_NAMES[label_index])
                        
            print(correct_labels)
            for gt_box, gt_label in zip(gt_boxes, correct_labels):
                total_boxes += 1
                iou_threshold = 0.5

                for pred_box, pred_label in zip(boxes, pred_classes):
                    iou = detect_utils.calculate_iou(gt_box, pred_box)

                    if iou > iou_threshold:
                        num_correct_boxes += 1
                        if gt_label == pred_label:
                            num_correct_labels += 1
                        break
            #image = detect_utils.draw_boxes(boxes, pred_classes, labels, image_np)
            #cv2.imshow("result", image)


    box_accuracy = num_correct_boxes / total_boxes
    label_accuracy = num_correct_labels / total_boxes

    print(f"Bounding Box Accuracy: {box_accuracy * 100:.2f}%")
    print(f"Label Accuracy: {label_accuracy * 100:.2f}%")


In [ ]:
folder_path = "models"
num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

model_name = f"model_{num_files}.pth"
model_path = os.path.join(folder_path, model_name)

# Save the model
torch.save(model.state_dict(), model_path)